<a href="https://colab.research.google.com/github/HarshitaBhatia27/Deep-Learning-/blob/main/Neural_N_W_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda,Compose
import matplotlib.pyplot as plt

In [ ]:
training_data= datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [ ]:
batch_size=64

train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)

for x,y in test_dataloader:
  print("Shape of X [N,C,H,W]: ",x.shape)
  print("Shape of y: ", y.dtype)
  break

Shape of X [N,C,H,W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.int64


In [ ]:
device= "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(     #initializing required parameters
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x= self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


model= NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-3)

In [ ]:
def train(dataloader,model,loss_fn,optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

#compute pred error
    pred=model(X)
    loss=loss_fn(pred,y)


    #backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 ==0:
      loss,current= loss.item(), batch* len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [ ]:
from re import S
def test(dataloader, model,loss_fn):
  size=len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss,correct=0,0
  with torch.no_grad():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred=model(X)
      test_loss +=loss_fn(pred,y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /=size
  print(f"test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs= 5
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------")
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
------------------------
loss: 2.295277 [    0/60000]
loss: 2.291167 [ 6400/60000]
loss: 2.268976 [12800/60000]
loss: 2.264192 [19200/60000]
loss: 2.245068 [25600/60000]
loss: 2.217705 [32000/60000]
loss: 2.227641 [38400/60000]
loss: 2.188094 [44800/60000]
loss: 2.173462 [51200/60000]
loss: 2.147216 [57600/60000]
test Error: 
 Accuracy: 52.1%, Avg loss: 2.136539 

Epoch 2
------------------------
loss: 2.140092 [    0/60000]
loss: 2.138929 [ 6400/60000]
loss: 2.073135 [12800/60000]
loss: 2.089347 [19200/60000]
loss: 2.040411 [25600/60000]
loss: 1.980618 [32000/60000]
loss: 2.008875 [38400/60000]
loss: 1.925804 [44800/60000]
loss: 1.915500 [51200/60000]
loss: 1.848373 [57600/60000]
test Error: 
 Accuracy: 61.0%, Avg loss: 1.844050 

Epoch 3
------------------------
loss: 1.875993 [    0/60000]
loss: 1.853450 [ 6400/60000]
loss: 1.725934 [12800/60000]
loss: 1.764314 [19200/60000]
loss: 1.656548 [25600/60000]
loss: 1.614288 [32000/60000]
loss: 1.636853 [38400/60000]
loss: 1.537608

In [ ]:
torch.save(model.state_dict(),"model.pth")
print("Saved Pytorch Model state to model.pth")

Saved Pytorch Model state to model.pth


In [ ]:
#load model
model= NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
#prediction
classes=[
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Sneaker",
    "Shirt",
    "Bag",
    "Ankle boot",
]
model.eval()
x,y = test_data[0][0],test_data[0][1]
with torch.no_grad():
  pred=model(x)
  predicted, actual= classes[pred[0].argmax(0)],classes[y]
  print(f'Predicted: " {predicted}", Actual: "{actual}"')

Predicted: " Ankle boot", Actual: "Ankle boot"
